In [39]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2

include("Data_Generation_Product_Design.jl")
include("Estimation_Product_Design.jl")
include("Product_Design_Models.jl")
include("Evaluation_Product_Design.jl")

calculate_profit (generic function with 1 method)

In [32]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
project_dir = dirname(grand_pa_dir)
data_dir = string(project_dir, "/Data/Product_Design/")
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)

Data directory: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Design/


In [40]:
string(data_dir,"obj_True_list.jld2")

"/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Design/obj_True_list.jld2"

In [41]:
save(string(data_dir,"obj_True_list.jld2"),obj_True_list) 

Error encountered while save File{DataFormat{:JLD2}, String}("/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Design/obj_True_list.jld2").

Fatal error:


CapturedException: must supply a name for each variable
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] fileio_save(f::File{DataFormat{:JLD2}, String}, value::Vector{Float64}; kwargs::@Kwargs{})
    @ JLD2 ~/.julia/packages/JLD2/9xHXH/src/fileio.jl:33
  [3] #invokelatest#2
    @ ./essentials.jl:892 [inlined]
  [4] invokelatest
    @ ./essentials.jl:889 [inlined]
  [5] action(call::Symbol, libraries::Vector{Union{Base.PkgId, Module}}, file::Formatted, args::Vector{Float64}; options::@Kwargs{})
    @ FileIO ~/.julia/packages/FileIO/ePyCW/src/loadsave.jl:219
  [6] action
    @ ~/.julia/packages/FileIO/ePyCW/src/loadsave.jl:196 [inlined]
  [7] action
    @ ~/.julia/packages/FileIO/ePyCW/src/loadsave.jl:185 [inlined]
  [8] save(file::String, args::Vector{Float64}; options::@Kwargs{})
    @ FileIO ~/.julia/packages/FileIO/ePyCW/src/loadsave.jl:129
  [9] save(file::String, args::Vector{Float64})
    @ FileIO ~/.julia/packages/FileIO/ePyCW/src/loadsave.jl:125
 [10] top-level 

In [38]:
obj_True_list

100-element Vector{Float64}:
 1.7801027682122357
 1.8356519378920155
 0.4739111223335175
 1.787921328075917
 1.822145644075634
 1.8369753608665573
 1.8229855035118034
 1.8385292016718233
 1.8301385003336976
 1.0766000582391881
 ⋮
 1.1328964408600934
 1.592896357876563
 0.1430869354433337
 1.2076807644326963
 1.6507171931472382
 1.8047365907048731
 0.6949994159231704
 0.7545134278738004
 0.9210849362232238

In [4]:
d = 5 # num of product feature
p = 10 # num of customer feature
n = 100 # num of training samples
n_test = 100
m = 5 # num of products
s = 10
Random.seed!(1)

TaskLocalRNG()

### Data Generation

In [5]:
theta_true, r_params = Generate_Wang_Qi_Max_True_Paras(d,p,s);

In [6]:
X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data(d, p, n, m,theta_true);

In [7]:
X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data(d, p, n_test, m,theta_true);

### Estimation

In [8]:
lambda = 0.0001
alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result = estimate_parameters(X_train,Y_train,Z_train,lambda, d, p, initial_theta=randn((d+1)*(p+1)) * 0.1);

### Oracle

In [9]:
r0 = r_params.r0;
r = r_params.r;
alp0_true = theta_true.alpha0;
alp_true = theta_true.alpha;
beta_true = theta_true.beta;
A_true = theta_true.A;

In [14]:
obj_True_list = zeros(n_test);
x_True_list = zeros(n_test,d);
time_True_list = zeros(n_test);
profit_True_list = zeros(n_test);
for i in 1:n_test
    z_input = Z_test[i,:];
    nu0 = alp0_true + beta_true' * z_input;
    nu = alp_true .+ A_true * z_input;
    # obj_True_list[i], x_True_list[i,:], time_True_list[i] = Product_Design_Ours_ETO(d,nu0, nu, r0, r, z_input);
    obj_True_list[i], x_True_list[i,:], time_True_list[i] = Product_Design_ETO(d,nu0, nu, r0, r, z_input);
    profit_True_list[i] = calculate_profit(alp0_true, alp_true, beta_true, A_true, r0, r, x_True_list[i,:], z_input)
    if i % 20 == 1
        println("i=$i, obj=$(round(obj_True_list[i], digits=4)), profit=$(round(profit_True_list[i], digits=4)),x=$(round.(x_True_list[i,:], digits=4)), time=$(round(time_True_list[i],digits=4))")
    end
end

i=1, obj=1.7801, profit=1.78,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0223
i=21, obj=1.2758, profit=1.2758,x=[1.0, 0.0, 1.0, 1.0, 1.0], time=0.0292
i=41, obj=1.6859, profit=1.6856,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0361
i=61, obj=1.8377, profit=1.8377,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.015
i=81, obj=0.5898, profit=0.5898,x=[1.0, 0.0, 1.0, 1.0, 1.0], time=0.0276


In [ ]:
Result_True = Dict("obj_True_list"=>obj_True_list, "x_True_list"=>x_True_list, "time_True_list"=>time_True_list, "profit_True_list"=>profit_True_list);
save(string(data_dir,"Result_True.jld2"),Result_True) 

### Estimate-then-optimize model

In [15]:
obj_ETO_list = zeros(n_test);
x_ETO_list = zeros(n_test,d);
time_ETO_list = zeros(n_test);
profit_ETO_list = zeros(n_test);
for i in 1:n_test
    z_input = Z_test[i,:];
    nu0 = alpha0_hat + beta_hat' * z_input;
    nu = alpha_hat .+ A_hat * z_input;
    # obj_ETO_list[i], x_ETO_list[i,:], time_ETO_list[i] = Product_Design_Ours_ETO(d,nu0, nu, r0, r, z_input);
    obj_ETO_list[i], x_ETO_list[i,:], time_ETO_list[i] = Product_Design_ETO(d,nu0, nu, r0, r, z_input);
    profit_ETO_list[i] = calculate_profit(alp0_true, alp_true, beta_true, A_true, r0, r, x_ETO_list[i,:], z_input)
    if i % 20 == 1
        println("i=$i, obj=$(round(obj_ETO_list[i], digits=4)), profit=$(round(profit_ETO_list[i], digits=4)),x=$(round.(x_ETO_list[i,:], digits=4)), time=$(round(time_ETO_list[i],digits=4))")
    end
end

i=1, obj=1.8321, profit=1.78,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0131
i=21, obj=1.7206, profit=1.0408,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0274
i=41, obj=1.8385, profit=1.6856,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.011
i=61, obj=1.8396, profit=1.8377,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0018
i=81, obj=0.6491, profit=0.2685,x=[1.0, 1.0, 0.0, 1.0, 1.0], time=0.0271


In [ ]:
println("Average Profit of True Model: ", mean(profit_True_list))
println("Average Profit of ETO Model: ", mean(profit_ETO_list))

Average Profit of True Model: 1.2409160158553614
Average Profit of ETO Model: 1.138143717247198


In [28]:
profit_ETO_list

100-element Vector{Float64}:
 1.7800069745910145
 1.8356300744491958
 0.473618581811355
 1.7878792071312535
 1.8220663892962063
 1.8369318550718645
 1.8229634030700606
 1.8385157950359103
 1.8301211710072538
 0.7334694306652048
 ⋮
 1.1194922947833637
 1.592759101609332
 0.01410995705639024
 0.888932256771347
 1.6504181505163082
 1.8046255209075819
 0.46448012062646904
 0.17154994191021017
 0.6217602526649578

### Robust Model

In [ ]:
gamma_list = [0.0,0.0001,0.01,0.1]
psi_lb = -100
psi_ub = 100
phi_lb = -100
phi_ub = 100

100

In [25]:
obj_RO_Dict = Dict(); x_RO_Dict = Dict(); time_RO_Dict = Dict(); profit_RO_Dict = Dict();
for gamma in gamma_list
    obj_RO_list = zeros(n_test);
    x_RO_list = zeros(n_test,d);
    time_RO_list = zeros(n_test);
    profit_RO_list = zeros(n_test);
    for i in 1:n_test
        z_input = Z_test[i,:];
        obj_RO_list[i], x_RO_list[i,:], time_RO_list[i] = Robust_Product_Design(2, d, p, gamma, psi_lb, psi_ub, phi_lb, phi_ub, alpha0_hat, alpha_hat, beta_hat, A_hat, r0, r, z_input);
        profit_RO_list[i] = calculate_profit(alp0_true, alp_true, beta_true, A_true, r0, r, x_RO_list[i,:], z_input)
        if i % 20 == 1
            println("gamma=$gamma, i=$i, obj=$(round(obj_RO_list[i], digits=4)), profit=$(round(profit_RO_list[i], digits=4)),x=$(round.(x_RO_list[i,:], digits=4)), time=$(round(time_RO_list[i],digits=4))")
        end
    end
    obj_RO_Dict["gamma=$gamma"] = obj_RO_list
    x_RO_Dict["gamma=$gamma"] = x_RO_list       
    time_RO_Dict["gamma=$gamma"] = time_RO_list
    profit_RO_Dict["gamma=$gamma"] = profit_RO_list
end

gamma=0.0, i=1, obj=1.8321, profit=1.78,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0299
gamma=0.0, i=21, obj=1.7204, profit=1.0408,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0257
gamma=0.0, i=41, obj=1.8385, profit=1.6856,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.026
gamma=0.0, i=61, obj=1.8395, profit=1.8377,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0314
gamma=0.0, i=81, obj=0.6491, profit=0.2685,x=[1.0, 1.0, 0.0, 1.0, 1.0], time=0.0261
gamma=0.0001, i=1, obj=1.8321, profit=1.78,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0292
gamma=0.0001, i=21, obj=1.7204, profit=1.0408,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0266
gamma=0.0001, i=41, obj=1.8385, profit=1.6856,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0298
gamma=0.0001, i=61, obj=1.8395, profit=1.8377,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.0312
gamma=0.0001, i=81, obj=0.649, profit=0.2685,x=[1.0, 1.0, 0.0, 1.0, 1.0], time=0.0271
gamma=0.01, i=1, obj=1.8318, profit=1.78,x=[1.0, 0.0, 0.0, 1.0, 1.0], time=0.029
gamma=0.01, i=21, obj=1.7157, profit=1.0408,x=[1.0, 0.0, 0.0, 1.0, 1.0],

In [26]:
for gamma in gamma_list
    println("gamma=$gamma, Average Profit of RO Model: ", mean(profit_RO_Dict["gamma=$gamma"]))
end

gamma=0.0, Average Profit of RO Model: 1.1381440576822428
gamma=0.0001, Average Profit of RO Model: 1.1381440576822428
gamma=0.01, Average Profit of RO Model: 1.1413311863074125
gamma=0.1, Average Profit of RO Model: 1.1398810719966246
